# Setup Environment
Use validation set images from ILSVRC 2012 Challenge in a google drive folder with their labels in a meta.json. (Using validation set since no meta data available for test set)

For more information see: http://image-net.org/challenges/LSVRC/2012/

In [ ]:
from torchvision.datasets.folder import default_loader
import json
import torch
from torch.autograd import Variable as V
import torchvision.models as tmodels
import torchvision.transforms as transforms
from torch.nn import functional as F
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models import *
import numpy as np
os.environ['TORCH_HOME'] = '/mnt/raid/ni/agnessa/RSA/.cache/torch/' #directory where pretrained models are saved
imagenet_validation_path = '/mnt/raid/data/ni/dnn/ILSVRC2012_img_val'
meta_file_path = '/mnt/raid/ni/agnessa/RSA/'
layers_path = '/mnt/raid/ni/agnessa/RSA/layer_names'
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/Objects/'


# Select Data and get Metadata
Select 10 images of each of the 1000 classes of the validation data set together with their label. 

In [ ]:
class ILSVRCSubDataset(Dataset):
    """ILSVRC 2012 subset of the original val dataset"""

    def __init__(self, json_file, root, transform=None):
        """
        Args:
            json_file (string): Path to the json file with meta.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root = root
        self.transform = transform
        
        # Parse META File
        with open(json_file, "r") as fd:
            self.meta = json.load(fd)
        print(self.meta)

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        path = os.path.join(self.root,
                            self.meta[idx]["0"]) #merge root and the filename of the sample
        sample = default_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
            
        wnid = self.meta[idx]["1"]
            
        return sample, wnid #sample, class

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset_val = ILSVRCSubDataset(json_file=os.path.join(meta_file_path,'meta.json'),
                               root=imagenet_validation_path,
                               transform=data_transforms)

dataloaders = torch.utils.data.DataLoader(dataset_val, #Combines a dataset and a sampler, and provides an iterable over the given dataset.
                                          batch_size=1, #how many samples per batch to load
                                          shuffle=False)


# Get the activations from a layer for all samples and save them
Use the subset with 10 images of 1000 classes on torchvisions pretrained models, get the activations of specific layers and calculate the Input RDM by correlating between the activations.

## Define functions to create filenames

In [ ]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"        

## Get activations

In [ ]:
# Iterator shouldn't be recreated every time, because it always returns the first element
# Which breaks everything if shuffling is disabled
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#load json file with the layers of interest 
model_name = 'vgg16'
json_file_layers=os.path.join(layers_path,model_name + '_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)

model = eval( model_name+'(pretrained=True)')
NR_OF_SAMPLES = 1 #num classes*num samples per class;  len(dataset_val)   
batch_size = 1

#important: put model in evaluation mode for consistent results
model.eval()
hooks = list()

for layer,m in model.named_modules():
    data_iterator = iter(dataloaders) #create an iterator for each layer
    activations = list() 
    m.auto_name = layer
    handle = m.register_forward_hook(lambda m, i, o: print(f"Hook called from: {m.auto_name}\n Input dimension: {i[0].size()} \n Output dimension: {o.shape} \n ############################")
    )
    hooks.append(handle)
    
    for i in range(int(NR_OF_SAMPLES/batch_size)): #for each batch get the activations
        print(".", end='')
        cur = next(data_iterator)[0] #cur: images, labels             
        out = model(cur) #probabilities of each class


